In [7]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers
from keras.callbacks import TensorBoard
import datetime

import numpy as np

Using TensorFlow backend.


In [8]:
def get_timestamp():
    return '{:%Y-%m-%d-%H-%M-%S}'.format(datetime.datetime.now())

In [9]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [10]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [11]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [24]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

datagen.fit(x_train)



In [22]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), padding = "same", activation='relu'))
small_cnn.add(Conv2D(64, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(128, (3, 3), padding = "same", activation='relu'))
small_cnn.add(Conv2D(128, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())

small_cnn.add(Dense(units=10, activation='softmax'))

In [23]:
#model = linear
model = small_cnn

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [12]:
n_train    = 6000
batch_size = 128
epochs     = 20

name = 'graph'

tensorboard = TensorBoard( log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
                          write_graph=True, write_images=False)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    #steps_per_epoch=len(x_train) / 32, epochs=epochs)

#model.fit_generator(datagen.flow(x_train[:6000], y_train[:6000], batch_size=batch_size),
          #steps_per_epoch = n_train / batch_size,
          #epochs=epochs,
          #verbose=1,
          #validation_data=(x_test[:1000], y_test[:1000]),
          #callbacks=[tensorboard])

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:1000], y_test[:1000]),
         callbacks=[tensorboard])

Train on 6000 samples, validate on 1000 samples
Epoch 1/20
6000/6000 [==============================] - 28s - loss: 2.2607 - acc: 0.1548 - val_loss: 2.0939 - val_acc: 0.3110
Epoch 2/20
6000/6000 [==============================] - 32s - loss: 2.0598 - acc: 0.2572 - val_loss: 1.9522 - val_acc: 0.3400
Epoch 3/20
6000/6000 [==============================] - 33s - loss: 1.9169 - acc: 0.3155 - val_loss: 1.7834 - val_acc: 0.3770
Epoch 4/20
6000/6000 [==============================] - 27s - loss: 1.7941 - acc: 0.3645 - val_loss: 1.7225 - val_acc: 0.3720
Epoch 5/20
6000/6000 [==============================] - 31s - loss: 1.7241 - acc: 0.3862 - val_loss: 1.7265 - val_acc: 0.3800
Epoch 6/20
6000/6000 [==============================] - 28s - loss: 1.6606 - acc: 0.4005 - val_loss: 1.6456 - val_acc: 0.4080
Epoch 7/20
6000/6000 [==============================] - 26s - loss: 1.6034 - acc: 0.4232 - val_loss: 1.7096 - val_acc: 0.4030
Epoch 8/20
6000/6000 [==============================] - 29s - loss: 1.

In [13]:
_, acc = model.evaluate(x_test[:1000], y_test[:1000], verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.509


In [28]:
n_train    = 6000
batch_size = 128
epochs     = 20

name = 'graph'

tensorboard = TensorBoard( log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit_generator(datagen.flow(x_train[:6000], y_train[:6000], batch_size=batch_size),
          steps_per_epoch = n_train / batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:1000], y_test[:1000]),
          callbacks=[tensorboard])

Epoch 1/20
47/46 [==============================] - 41s - loss: 2.2837 - acc: 0.1404 - val_loss: 2.2326 - val_acc: 0.1420
Epoch 2/20
47/46 [==============================] - 34s - loss: 2.1740 - acc: 0.1994 - val_loss: 2.0694 - val_acc: 0.2610
Epoch 3/20
47/46 [==============================] - 36s - loss: 2.0924 - acc: 0.2289 - val_loss: 1.9644 - val_acc: 0.2860
Epoch 4/20
47/46 [==============================] - 39s - loss: 2.0325 - acc: 0.2496 - val_loss: 1.9738 - val_acc: 0.3130
Epoch 5/20
47/46 [==============================] - 32s - loss: 1.9765 - acc: 0.2882 - val_loss: 1.8387 - val_acc: 0.3360
Epoch 6/20
47/46 [==============================] - 37s - loss: 1.9149 - acc: 0.3045 - val_loss: 1.8000 - val_acc: 0.3650
Epoch 7/20
47/46 [==============================] - 35s - loss: 1.8595 - acc: 0.3166 - val_loss: 1.8841 - val_acc: 0.3270
Epoch 8/20
47/46 [==============================] - 42s - loss: 1.8426 - acc: 0.3253 - val_loss: 1.8218 - val_acc: 0.3340
Epoch 9/20
47/46 [======

In [29]:
_, acc = model.evaluate(x_test[:1000], y_test[:1000], verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.497
